In [1]:
# !pip install protobuf=3.20.2
!pip install protobuf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
import flair.datasets
corpus = flair.datasets.UD_INDONESIAN()
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings
from typing import List

C:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-19 13:40:11,991 Reading data from C:\Users\akhyar\.flair\datasets\ud_indonesian
2024-10-19 13:40:11,992 Train: C:\Users\akhyar\.flair\datasets\ud_indonesian\id_gsd-ud-train.conllu
2024-10-19 13:40:11,993 Dev: C:\Users\akhyar\.flair\datasets\ud_indonesian\id_gsd-ud-dev.conllu
2024-10-19 13:40:11,995 Test: C:\Users\akhyar\.flair\datasets\ud_indonesian\id_gsd-ud-test.conllu


ImportError: cannot import name 'BertEmbeddings' from 'flair.embeddings' (C:\Users\akhyar\AppData\Roaming\Python\Python311\site-packages\flair\embeddings\__init__.py)

In [ ]:
# 1. get the corpus
# corpus = NLPTaskDataFetcher.load_corpus(NLPTask.UD_INDONESIAN)

# 2. what tag do we want to predict?
tag_type = 'upos'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),
    #WordEmbeddings('glove'),
    #BertEmbeddings('bert-base-multilingual-cased')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-universal-pos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

In [ ]:
from flair.data import Sentence

sentence = Sentence('saya dan dia kemarin pergi ke pasar bersama untuk membeli jeruk')
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())